## 🔧 Install Dependencies
We need the `google-generativeai` SDK to access Gemini models.

In [1]:
!pip install -q google-generativeai


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: C:\Users\HP\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## 🔑 Configure API Key
You’ll need a Google AI Studio API key.  
1. Go to [Google AI Studio](https://aistudio.google.com/)  
2. Generate an API key  
3. Store it in your environment


In [2]:
import os
import json
import google.generativeai as genai

from dotenv import load_dotenv
load_dotenv()

# Set your API key (make sure you stored it as environment variable)
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 📋 List Available Models
This helps us see what Gemini models are accessible.


In [3]:
models = genai.list_models()
for m in models:
    print(m.name, "\n- supports ", m.supported_generation_methods)
    print()

models/embedding-gecko-001 
- supports  ['embedText', 'countTextTokens']

models/gemini-2.5-pro-preview-03-25 
- supports  ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

models/gemini-2.5-flash-preview-05-20 
- supports  ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

models/gemini-2.5-flash 
- supports  ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

models/gemini-2.5-flash-lite-preview-06-17 
- supports  ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

models/gemini-2.5-pro-preview-05-06 
- supports  ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

models/gemini-2.5-pro-preview-06-05 
- supports  ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

models/gemini-2.5-pro 
- supports  ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']

models/gemini-2.0-f

## ✍️ Basic Text Generation
We’ll use `gemini-2.5-flash` for a simple text generation task.

In [4]:
model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Write a short poem about autumn leaves.")
print(response.text)


A fiery blush, a golden gleam,
They dance and fall, a vibrant dream.
From branches high, a gentle flight,
Then carpet earth in russet light.


## 🎛 Playing with Parameters
Gemini supports multiple parameters:
- **temperature** → randomness (0 = deterministic, 1 = creative)  
- **top_p** → nucleus sampling, picks top p probability tokens
- **top_k** → how many tokens to sample from
- **max_output_tokens** → maximum response token length

In [5]:
prompt = "Explain black holes to a 10-year-old."

response = model.generate_content(
    prompt,
    generation_config={
        "temperature": 0.9,
        "top_p": 0.8,
        "top_k": 40,
        "max_output_tokens": 256 # Try increasing the limit
    }
)
# print(response.text)

if response.candidates and response.candidates[0].content.parts:
    print(response.candidates[0].content.parts[0].text)
else:
    print("No text returned. Finish reason:", response.candidates[0].finish_reason)

No text returned. Finish reason: FinishReason.MAX_TOKENS


## 📦 JSON Structured Output
We can request structured responses (useful for apps!).

### 1. Without Pydantic

In [6]:
prompt = "Return 3 Indian states and their capitals in JSON format."

response = model.generate_content(
    prompt,
    generation_config={
        "response_mime_type": "application/json"
    }
)
print(response.text)
print(type(response.text))

json_response = json.loads(response.text)
print(json_response)
print(type(json_response))

[
  {
    "state": "Maharashtra",
    "capital": "Mumbai"
  },
  {
    "state": "Karnataka",
    "capital": "Bengaluru"
  },
  {
    "state": "Rajasthan",
    "capital": "Jaipur"
  }
]
<class 'str'>
[{'state': 'Maharashtra', 'capital': 'Mumbai'}, {'state': 'Karnataka', 'capital': 'Bengaluru'}, {'state': 'Rajasthan', 'capital': 'Jaipur'}]
<class 'list'>


### 2. With Pydantic

In [7]:
from pydantic import BaseModel
from typing import List, Dict, Any

class StateCapital(BaseModel):
    state: str
    capital: str
    food: str

class StatesResponse(BaseModel):
    states: List[StateCapital]

    def to_gemini_schema(self) -> Dict[str, Any]:
        """
        Convert this Pydantic model's JSON Schema into Gemini's
        response_schema (OpenAPI-subset) format with all $ref's
        dereferenced and only supported fields preserved.
        """
        raw = self.model_json_schema()
        return self._convert_schema(raw, root=raw)

    _ALLOWED_KEYS = {
        "type", "format", "properties", "required", "items", "enum",
        "maxItems", "minItems", "maximum", "minimum", "nullable", "anyOf"
    }

    def _resolve_ref(self, ref: str, root: Dict[str, Any]) -> Dict[str, Any]:
        """Resolve a local JSON Pointer like '#/$defs/Foo' or '#/definitions/Foo'."""
        if not ref.startswith("#/"):
            return {}
        node: Dict[str, Any] = root
        for part in ref[2:].split("/"):
            node = node.get(part, {})
        return node

    def _strip_unsupported(self, schema: Dict[str, Any]) -> Dict[str, Any]:
        """Remove fields the Gemini response schema ignores."""
        return {k: v for k, v in schema.items() if k in self._ALLOWED_KEYS}

    # CORE CONVERSION LOGIC
    def _convert_schema(self, node: Dict[str, Any], root: Dict[str, Any]) -> Dict[str, Any]:
        # 1) Resolve $ref if present
        if "$ref" in node:
            node = self._resolve_ref(node["$ref"], root)

        # 2) Handle Optional/union encoded as anyOf[..., {"type": "null"}]
        if "anyOf" in node:
            variants = node["anyOf"]
            nullable = any(v.get("type") == "null" for v in variants)
            non_null = [v for v in variants if v.get("type") != "null"]
            if len(non_null) == 1:
                base = self._convert_schema(non_null[0], root)
                if nullable:
                    base["nullable"] = True
                return base
            # General union: keep anyOf (Gemini supports anyOf)
            return {"anyOf": [self._convert_schema(v, root) for v in non_null]}

        typ = node.get("type")

        # 3) Objects
        if typ == "object" or "properties" in node:
            out: Dict[str, Any] = {"type": "object", "properties": {}}
            if "required" in node:
                out["required"] = list(node["required"])
            for name, prop in (node.get("properties") or {}).items():
                out["properties"][name] = self._convert_schema(prop, root)
            # nullable object if Pydantic put it at this level (rare)
            if node.get("nullable") is True:
                out["nullable"] = True
            return self._strip_unsupported(out)

        # 4) Arrays
        if typ == "array":
            items = self._convert_schema(node.get("items", {}), root)
            arr: Dict[str, Any] = {"type": "array", "items": items}
            if "minItems" in node:
                arr["minItems"] = node["minItems"]
            if "maxItems" in node:
                arr["maxItems"] = node["maxItems"]
            return self._strip_unsupported(arr)

        # 5) Primitives
        if typ in {"string", "integer", "number", "boolean"}:
            leaf: Dict[str, Any] = {"type": typ}
            # carry through supported constraints if present
            for k in ("format", "enum", "minimum", "maximum"):
                if k in node:
                    leaf[k] = node[k]
            if node.get("nullable") is True:
                leaf["nullable"] = True
            return self._strip_unsupported(leaf)

        # 6) Fallback (treat as string)
        return {"type": "string"}


In [8]:
from pprint import pprint

response_format = StatesResponse(states=[StateCapital(state="Tamil Nadu", capital="Chennai", food="Dosa"),])

# Default pydantic schema
print("Pydantic schema:")
pprint(response_format.model_json_schema())
print("-"*100)

# Gemini-compatible schema
print("Gemini-compatible schema:")
pprint(response_format.to_gemini_schema())

Pydantic schema:
{'$defs': {'StateCapital': {'properties': {'capital': {'title': 'Capital',
                                                       'type': 'string'},
                                           'food': {'title': 'Food',
                                                    'type': 'string'},
                                           'state': {'title': 'State',
                                                     'type': 'string'}},
                            'required': ['state', 'capital', 'food'],
                            'title': 'StateCapital',
                            'type': 'object'}},
 'properties': {'states': {'items': {'$ref': '#/$defs/StateCapital'},
                           'title': 'States',
                           'type': 'array'}},
 'required': ['states'],
 'title': 'StatesResponse',
 'type': 'object'}
----------------------------------------------------------------------------------------------------
Gemini-compatible schema:
{'properties': {'s

In [9]:
prompt = "Return 3 Indian states and their capitals in JSON format."

response = model.generate_content(
    prompt,
    generation_config={
        "response_mime_type": "application/json",
        "response_schema": response_format.to_gemini_schema()
    }
)

print(json.loads(response.text))

{'states': [{'state': 'Maharashtra', 'capital': 'Mumbai', 'food': 'Vada Pav'}, {'state': 'Rajasthan', 'capital': 'Jaipur', 'food': 'Dal Bati Churma'}, {'state': 'Karnataka', 'capital': 'Bengaluru', 'food': 'Bisi Bele Bath'}]}


## 💬 Multi-Turn Chat
Gemini supports chat-like interactions where history is preserved.

In [10]:
chat = model.start_chat(history=[])

r1 = chat.send_message("Hello, who are you?")
print("Bot:", r1.text)
print("-"*100)

r2 = chat.send_message("Can you explain quantum computing simply?")
print("Bot:", r2.text)
print("-"*100)

r3 = chat.send_message("What was the first question I asked you?")
print("Bot:", r3.text)

Bot: Hello! I am a large language model, trained by Google.

I'm here to provide information, answer your questions, generate creative text, and assist you with various tasks through conversation.

How can I help you today?
----------------------------------------------------------------------------------------------------
Bot: Imagine a regular computer as a light switch that can be either **ON (1)** or **OFF (0)**. It processes information by flipping these switches one by one.

Quantum computing is like having a light switch that can be **ON**, **OFF**, and also **ON and OFF at the same time** (in a blurry, "maybe" state). And what's more, these "maybe" switches can influence each other in strange ways.

Let's break it down into three key ideas:

1.  **Qubits (Quantum Bits): The "Maybe" Switch**
    *   **Classical Bit:** Is either a 0 or a 1. Like a coin lying flat, showing either heads or tails.
    *   **Quantum Qubit:** Can be a 0, a 1, *or both at the same time* (this is called

In [11]:
chat.history

[parts {
   text: "Hello, who are you?"
 }
 role: "user",
 parts {
   text: "Hello! I am a large language model, trained by Google.\n\nI\'m here to provide information, answer your questions, generate creative text, and assist you with various tasks through conversation.\n\nHow can I help you today?"
 }
 role: "model",
 parts {
   text: "Can you explain quantum computing simply?"
 }
 role: "user",
 parts {
   text: "Imagine a regular computer as a light switch that can be either **ON (1)** or **OFF (0)**. It processes information by flipping these switches one by one.\n\nQuantum computing is like having a light switch that can be **ON**, **OFF**, and also **ON and OFF at the same time** (in a blurry, \"maybe\" state). And what\'s more, these \"maybe\" switches can influence each other in strange ways.\n\nLet\'s break it down into three key ideas:\n\n1.  **Qubits (Quantum Bits): The \"Maybe\" Switch**\n    *   **Classical Bit:** Is either a 0 or a 1. Like a coin lying flat, showing eith

## 🛡 Safety Settings
We can adjust safety thresholds for categories like hate speech, dangerous content, etc.

Some supported categories are:
- `HARM_CATEGORY_HARASSMENT`
- `HARM_CATEGORY_HATE_SPEECH`
- `HARM_CATEGORY_SEXUALLY_EXPLICIT`
- `HARM_CATEGORY_DANGEROUS_CONTENT`
- `HARM_CATEGORY_CIVIC_INTEGRITY`

In [12]:
response = model.generate_content(
    "Give me a recipe for chocolate cake",
    safety_settings=[
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_ONLY_HIGH"
        }
    ]
)
print(response.text)


This is a fantastic, classic chocolate cake recipe that's incredibly moist, rich, and perfect for any occasion. It's often called "Wacky Cake" or "Depression-Era Cake" because it uses oil instead of butter and no eggs, yet still produces a phenomenal result. I'll include a simple, delicious chocolate buttercream frosting to go with it.

---

## The Best Moist Chocolate Cake with Chocolate Buttercream

This cake is surprisingly easy to make and yields a deeply chocolatey, super moist crumb. The hot coffee isn't for flavor (you won't taste it!), but it helps "bloom" the cocoa powder, intensifying the chocolate flavor and contributing to the cake's tender texture.

**Yields:** One 9x13 inch sheet cake OR two 8 or 9-inch round cake layers (for a layer cake)
**Prep time:** 20 minutes
**Cook time:** 30-35 minutes (for rounds), 35-40 minutes (for sheet cake)

---

### **Part 1: The Chocolate Cake**

**Ingredients:**

*   2 cups (260g) All-Purpose Flour
*   2 cups (400g) Granulated Sugar
*   ¾

## 🧠 Reasoning Mode (Experimental)
Some Gemini models (e.g., `gemini-2.0-flash-thinking-exp-1219`) support reasoning with intermediate steps.

In [13]:
reasoning_model = genai.GenerativeModel("gemini-2.0-flash-thinking-exp-1219")

resp = reasoning_model.generate_content(
    "Solve: If a train leaves at 2 PM at 60 km/h and another at 3 PM at 90 km/h, when do they meet?",
    generation_config={
        "temperature": 0.2
    }
)
print(resp.text)


Here's how to solve this problem:

1.  **Calculate Train 1's head start:**
    Train 1 leaves at 2 PM and Train 2 leaves at 3 PM. This means Train 1 travels for 1 hour before Train 2 even starts.
    Distance covered by Train 1 in that hour = Speed × Time = 60 km/h × 1 h = 60 km.
    So, at 3 PM, Train 1 is 60 km ahead of Train 2.

2.  **Determine the relative speed:**
    Both trains are traveling in the same direction, but Train 2 is faster. The rate at which Train 2 closes the gap on Train 1 is their difference in speed.
    Relative speed = Speed of Train 2 - Speed of Train 1
    Relative speed = 90 km/h - 60 km/h = 30 km/h.
    This means Train 2 gains 30 km on Train 1 every hour.

3.  **Calculate the time it takes for Train 2 to catch up:**
    Train 2 needs to close a 60 km gap at a relative speed of 30 km/h.
    Time = Distance / Relative Speed
    Time = 60 km / 30 km/h = 2 hours.

4.  **Find the meeting time:**
    Train 2 starts at 3 PM and takes 2 hours to catch up.
    Mee